In [ ]:
import csv
import logging
import sys

import numpy as np
import matplotlib.pyplot as plt

from numpy.polynomial import Polynomial

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s : %(message)s',
    stream=sys.stderr,
)

#### Parse functions

In [ ]:
def parse_coeffs(row: list, header: dict[str, int]) -> tuple[list[list[float]], list[list[float]]]:
    a = []
    b = []
    for i in range(6):
        a.append([row[header[f"a{i}{j}"]] for j in range(9)])
    for i in range(2):
        b.append([row[header[f"b{i}{j}"]] for j in range(9)])
    return a, b


def load_polynomes(filename: str) -> tuple[list[list[list[float]]], list[list[list[float]]]]:
    with open(filename) as f:
        reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
        header = None
        a_coeffs = []
        b_coeffs = []
        for row in reader:
            if header is None:
                header = {name: i for i, name in enumerate(row)}
                continue
            a, b = parse_coeffs(row, header)
            a_coeffs.append(a)
            b_coeffs.append(b)
    return a_coeffs, b_coeffs


def join_polynomes(a_coeffs):
    x_s = []
    y_s = []
    for i in range(len(a_coeffs)):
        a = a_coeffs[i]
        if not any(a):
            continue
        a_poly = Polynomial(a)
        x = np.arange(0, 1, 0.01)
        y = a_poly(x)
        x += i
        x_s.append(x)
        y_s.append(y)
    x_s = np.concat(x_s)
    y_s = np.concat(y_s)
    return x_s, y_s

In [ ]:
a_coeffs_st, b_coeffs_st = load_polynomes("response_output_hpge_point.csv")
a_coeffs_mt, b_coeffs_mt = load_polynomes("response_output_hpge_point_mt.csv")
# a_coeffs_mt, b_coeffs_mt = load_polynomes("response_output_scintil_point.csv")

In [ ]:
x_s, y_s1 = join_polynomes(a_coeffs_st[2])
x_s, y_s2 = join_polynomes(a_coeffs_mt[2])
plt.scatter(x_s, y_s1)
plt.scatter(x_s, y_s2)

In [ ]:
plt.scatter(x_s, y_s1 - y_s2)

In [ ]:
x_s, y_s1 = join_polynomes(b_coeffs_st[1])
x_s, y_s2 = join_polynomes(b_coeffs_mt[1])
plt.scatter(x_s, y_s1)
plt.scatter(x_s, y_s2)

#### Compare function

In [ ]:
EPS = 1e-10

def print_max_diff(y1: np.ndarray, y2: np.ndarray):
    abs_diff = np.abs(y1 - y2)
    max_abs_diff_idx = np.argmax(abs_diff)
    max_abs_diff = abs_diff[max_abs_diff_idx]
    max_rel_diff = 2 * max_abs_diff / (y1[max_abs_diff_idx] + y2[max_abs_diff_idx] + EPS)
    logging.error(f"    idx={max_abs_diff_idx}: abs_diff={max_abs_diff:.4}, rel_diff={max_rel_diff:.3}")
    

def compare2coeffs(coeff_1: list[float], coeff_2: list[float], epsilon: float) -> bool:
    assert len(coeff_1) == len(coeff_2)
    p1 = Polynomial(coeff_1)
    p2 = Polynomial(coeff_2)
    x = np.linspace(0, 1, 20)
    y1 = p1(x)
    y2 = p2(x)
    res = np.allclose(y1, y2, rtol=epsilon)
    if not res:
        print_max_diff(y1, y2)
    else:
        logging.debug("    OK")
    
    return np.allclose(y1, y2, rtol=epsilon)


def compare2points(point_1: list[list[float]], point_2: list[list[float]], epsilon: float, continue_after_fail: bool) -> bool:
    assert len(point_1) == len(point_2)
    main_res = True
    for coeff_num, (coeff_1, coeff_2) in enumerate(zip(point_1, point_2)):
        logging.debug(f"  coeff #{coeff_num}")
        res = compare2coeffs(coeff_1, coeff_2, epsilon)
        if not res:
            main_res = False
            if not continue_after_fail:
                return False
    return main_res


def compare2response(coeff_list_1: list[list[list[float]]], coeff_list_2: list[list[list[float]]], epsilon: float, continue_after_fail: bool = False) -> bool:
    assert len(coeff_list_1) == len(coeff_list_2)
    main_res = True
    for resp_point, (point_1, point_2) in enumerate(zip(coeff_list_1, coeff_list_2)):
        logging.debug(f"response point #{resp_point}")
        res = compare2points(point_1, point_2, epsilon, continue_after_fail)
        if not res:
            main_res = False
            if not continue_after_fail:
                return False
    return main_res

In [ ]:
print(compare2response(a_coeffs_st, a_coeffs_mt, epsilon=10**-8, continue_after_fail=True))